##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# SavedModel 마이그레이션 워크플로

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/migrate/saved_model"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서보기</a></td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/guide/migrate/saved_model.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a> </td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/guide/migrate/saved_model.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/guide/migrate/saved_model.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a>   </td>
</table>

모델을 TensorFlow 1의 그래프 및 세션에서 TensorFlow 2 API(예: `tf.function`, `tf.Module` 및 `tf.keras.Model)`로 마이그레이션한 후, 모델을 저장하고 로드하는 코드를 마이그레이션할 수 있습니다. 이 노트북은 TensorFlow 1 및 TensorFlow 2에서 SavedModel 형식으로 저장하고 로드할 수 있는 방법에 대한 예제를 제공합니다. 다음은 TensorFlow 1에서 TensorFlow 2로 마이그레이션할 때 사용하는 API 변경 사항에 대한 간략한 개요입니다.

 | TensorFlow 1 | TensorFlow 2로 마이그레이션
--- | --- | ---
**저장** | `tf.compat.v1.saved_model.Builder`<br>`tf.compat.v1.saved_model.simple_save` | `tf.saved_model.save`<br>Keras: `tf.keras.models.save_model`
**로드** | `tf.compat.v1.saved_model.load` | `tf.saved_model.load`<br>Keras: `tf.keras.models.load_model`
**서명**: 다음을 <br>실행하는 데<br>사용할 수 있는<br>입력 및 출력 텐서 집합 | `*.signature_def` 유틸을 사용하여 생성됨<br>(예: `tf.compat.v1.saved_model.predict_signature_def`) | `tf.function`을 작성하고 `tf.saved_model.save`의 <code>signatures</code> 인수<br>를 사용하여 내보내기 수행
**분류<br>및 회귀**:<br>서명의 특수한 유형 | <code>tf.compat.v1.saved_model.classification_signature_def</code>,<br><code>tf.compat.v1.saved_model.regression_signature_def</code> 및<br> 특정 Estimator 내보내기로<br> 생성 | 이 두 서명 유형은 TensorFlow 2에서 제거됨.<br>서빙 라이브러리에 이러한 메서드 이름이 필요한 경우<br>`tf.compat.v1.saved_model.signature_def_utils.MethodNameUpdater` 사용

매핑에 대한 자세한 설명은 아래의 [TensorFlow 1에서 TensorFlow 2로의 변경 사항](#changes_from_tf1_to_tf2) 섹션을 참조하세요.

## 설치하기

아래 예제는 TensorFlow 1 및 TensorFlow 2 API를 사용하여 동일한 더미 TensorFlow 모델(아래에서는 `add_two`로 정의)을 SavedModel 형식으로 내보내고 로드하는 방법을 보여줍니다. 가져오기 및 유틸리티 함수를 설정하는 것으로 시작합니다.

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1
import shutil

def remove_dir(path):
  try:
    shutil.rmtree(path)
  except:
    pass

def add_two(input):
  return input + 2

## TensorFlow 1: SavedModel 저장 및 내보내기

TensorFlow 1에서는 `tf.compat.v1.saved_model.Builder`, `tf.compat.v1.saved_model.simple_save` 및 `tf.estimator.Estimator.export_saved_model` API를 사용하여 TensorFlow 그래프 및 세션을 빌드하고 저장하고 내보냅니다.

### 1. SavedModelBuilder를 사용하여 그래프를 SavedModel로 저장하기

In [ ]:
remove_dir("saved-model-builder")

with tf.Graph().as_default() as g:
  with tf1.Session() as sess:
    input = tf1.placeholder(tf.float32, shape=[])
    output = add_two(input)
    print("add two output: ", sess.run(output, {input: 3.}))

    # Save with SavedModelBuilder
    builder = tf1.saved_model.Builder('saved-model-builder')
    sig_def = tf1.saved_model.predict_signature_def(
        inputs={'input': input},
        outputs={'output': output})
    builder.add_meta_graph_and_variables(
        sess, tags=["serve"], signature_def_map={
            tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY: sig_def
    })
    builder.save()

In [ ]:
!saved_model_cli run --dir saved-model-builder --tag_set serve \
 --signature_def serving_default --input_exprs input=10

### 2. 제공할 SavedModel 빌드하기

In [ ]:
remove_dir("simple-save")

with tf.Graph().as_default() as g:
  with tf1.Session() as sess:
    input = tf1.placeholder(tf.float32, shape=[])
    output = add_two(input)
    print("add_two output: ", sess.run(output, {input: 3.}))

    tf1.saved_model.simple_save(
        sess, 'simple-save',
        inputs={'input': input},
        outputs={'output': output})

In [ ]:
!saved_model_cli run --dir simple-save --tag_set serve \
 --signature_def serving_default --input_exprs input=10

### 3. Estimator 추론 그래프를 SavedModel로 내보내기

Estimator `model_fn`의 정의(아래에서 정의됨)에서 `tf.estimator.EstimatorSpec`에서 `export_outputs`를 반환하여 모델의 서명을 정의할 수 있습니다. 출력 유형은 다음과 같이 다양합니다.

- `tf.estimator.export.ClassificationOutput`
- `tf.estimator.export.RegressionOutput`
- `tf.estimator.export.PredictOutput`

이들은 각각 분류, 회귀, 예측 서명 유형을 생성합니다.

Estimator를 `tf.estimator.Estimator.export_saved_model`과 함께 내보내면 이러한 서명이 모델과 함께 저장됩니다.

In [ ]:
def model_fn(features, labels, mode):
  output = add_two(features['input'])
  step = tf1.train.get_global_step()
  return tf.estimator.EstimatorSpec(
      mode,
      predictions=output,
      train_op=step.assign_add(1),
      loss=tf.constant(0.),
      export_outputs={
          tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY: \
          tf.estimator.export.PredictOutput({'output': output})})
est = tf.estimator.Estimator(model_fn, 'estimator-checkpoints')

# Train for one step to create a checkpoint.
def train_fn():
  return tf.data.Dataset.from_tensors({'input': 3.})
est.train(train_fn, steps=1)

# This utility function `build_raw_serving_input_receiver_fn` takes in raw
# tensor features and builds an "input serving receiver function", which
# creates placeholder inputs to the model.
serving_input_fn = tf.estimator.export.build_raw_serving_input_receiver_fn(
    {'input': tf.constant(3.)})  # Pass in a dummy input batch.
estimator_path = est.export_saved_model('exported-estimator', serving_input_fn)

# Estimator's export_saved_model creates a time stamped directory. Move this
# to a set path so it can be inspected with `saved_model_cli` in the cell below.
!rm -rf estimator-model
import shutil
shutil.move(estimator_path, 'estimator-model')

In [ ]:
!saved_model_cli run --dir estimator-model --tag_set serve \
 --signature_def serving_default --input_exprs input=[10]

## TensorFlow 2: SavedModel 저장 및 내보내기

### tf.Module로 정의한 SavedModel 저장 및 내보내기

TensorFlow 2에서 모델을 내보내려면 `tf.Module` 또는 `tf.keras.Model`을 정의하여 모델의 모든 변수와 함수를 보유해야 합니다. 그러면 `tf.saved_model.save`를 호출하여 SavedModel을 생성할 수 있습니다. 자세한 내용은 [SavedModel 형식 사용하기](../saved_model.ipynb) 가이드의 *사용자 정의 모델 저장하기* 섹션을 참조하세요.

In [ ]:
class MyModel(tf.Module):
  @tf.function
  def __call__(self, input):
    return add_two(input)

model = MyModel()

@tf.function
def serving_default(input):
  return {'output': model(input)}

signature_function = serving_default.get_concrete_function(
    tf.TensorSpec(shape=[], dtype=tf.float32))
tf.saved_model.save(
    model, 'tf2-save', signatures={
        tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY: signature_function})

In [ ]:
!saved_model_cli run --dir tf2-save --tag_set serve \
 --signature_def serving_default --input_exprs input=10

### Keras로 정의한 SavedModel 저장 및 내보내기

사용이 중단: Keras 객체의 경우, [여기](https://www.tensorflow.org/guide/keras/save_and_serialize) 가이드에 설명된 대로 새로운 상위 수준 `.keras` 형식과 `tf.keras.Model.export`를 사용하는 것이 좋습니다. 기존 코드에는 하위 수준 SavedModel 형식이 계속 지원됩니다

`Mode.save` 또는 `tf.keras.models.save_model`와 같은 저장 및 내보내기용 Keras API는 `tf.keras.Model`에서 SavedModel을 내보낼 수 있습니다. 자세한 내용은 [Keras 모델 저장 및 로드하기](../..guide/keras/save_and_serialize)를 확인하세요.

In [ ]:
inp = tf.keras.Input(3)
out = add_two(inp)
model = tf.keras.Model(inputs=inp, outputs=out)

@tf.function(input_signature=[tf.TensorSpec(shape=[], dtype=tf.float32)])
def serving_default(input):
  return {'output': model(input)}

model.save('keras-model', save_format='tf', signatures={
        tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY: serving_default})

In [ ]:
!saved_model_cli run --dir keras-model --tag_set serve \
 --signature_def serving_default --input_exprs input=10

## SavedModel 로드하기

위의 API로 저장한 SavedModel은 TensorFlow 1 또는 TensorFlow 2 API를 사용하여 로드할 수 있습니다.

TensorFlow 1 SavedModel은 일반적으로 TensorFlow 2에 로드될 때 추론에 사용할 수 있지만 훈련(그래디언트 생성)은 SavedModel이 *리소스 변수*를 포함하는 경우에만 가능합니다. 변수의 dtype을 확인할 수 있습니다. 변수 dtype에 "_ref"가 포함되어 있으면 참조 변수입니다.

TensorFlow 2 SavedModel은 SavedModel을 서명과 함께 저장한는 한 TensorFlow 1에서 로드하고 실행할 수 있습니다.

아래 섹션에는 이전 섹션에서 저장한 SavedModels을 로드하고, 내보낸 서명을 호출하는 방법을 보여주는 코드 샘플이 포함되어 있습니다.

### TensorFlow 1: tf.saved_model.load로 저장한 모델 로드하기

TensorFlow 1에서는 `tf.saved_model.load`를 사용하여 SavedModel을 현재 그래프 및 세션으로 직접 가져올 수 있습니다. 다음과 같이 텐서 입력 및 출력 이름에서 `Session.run`을 호출할 수 있습니다.

In [ ]:
def load_tf1(path, input):
  print('Loading from', path)
  with tf.Graph().as_default() as g:
    with tf1.Session() as sess:
      meta_graph = tf1.saved_model.load(sess, ["serve"], path)
      sig_def = meta_graph.signature_def[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY]
      input_name = sig_def.inputs['input'].name
      output_name = sig_def.outputs['output'].name
      print('  Output with input', input, ': ', 
            sess.run(output_name, feed_dict={input_name: input}))

load_tf1('saved-model-builder', 5.)
load_tf1('simple-save', 5.)
load_tf1('estimator-model', [5.])  # Estimator's input must be batched.
load_tf1('tf2-save', 5.)
load_tf1('keras-model', 5.)

### TensorFlow 2: tf.saved_model로 저장한 모델 로드하기

TensorFlow 2에서 객체는 변수와 함수를 저장하는 Python 객체에 로드됩니다. 이것은 TensorFlow 1에서 저장한 모델과 호환됩니다.

자세한 내용은 [SavedModel 형식 사용하기](../..guide/saved_model) 가이드의 [사용자 정의 모델 로드 및 사용하기](../../guide/saved_model#loading_and_using_a_custom_model) 섹션과 `tf.saved_model.load` API 문서를 확인하세요.

In [ ]:
def load_tf2(path, input):
  print('Loading from', path)
  loaded = tf.saved_model.load(path)
  out = loaded.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY](
      tf.constant(input))['output']
  print('  Output with input', input, ': ', out)

load_tf2('saved-model-builder', 5.)
load_tf2('simple-save', 5.)
load_tf2('estimator-model', [5.])  # Estimator's input must be batched.
load_tf2('tf2-save', 5.)
load_tf2('keras-model', 5.)

TensorFlow 2 API로 저장한 모델은 모델에 연결된 `tf.function` 및  변수(서명으로 내보낸 항목 대신)에도 액세스할 수 있습니다. 예제는 다음과 같습니다.

In [ ]:
loaded = tf.saved_model.load('tf2-save')
print('restored __call__:', loaded.__call__)
print('output with input 5.', loaded(5))

### TensorFlow 2: Keras로 저장한 모델 로드하기

사용이 중단됨: Keras 객체의 경우, [여기](https://www.tensorflow.org/guide/keras/save_and_serialize) 가이드에 설명된 대로 새로운 상위 수준 `.keras` 형식과 `tf.keras.Model.export`를 사용하는 것이 좋습니다. 기존 코드에는 하위 수준 SavedModel 형식이 계속 지원됩니다

`tf.keras.models.load_model`과 같이 Keras로 로드하는 API를 사용하면 저장된 모델을 Keras 모델 객체로 다시 로드할 수 있습니다. Keras(`Model.save` 또는 `tf.keras.models.save_model`)로 저장한 SavedModels 만 로드할 수 있습니다.

`tf.saved_model.save`로 저장한 모델은 `tf.saved_model.load`로 로드되어야 합니다. `tf.saved_model.load`를 사용하여 `Model.save`로 저장한 Keras 모델은 로드할 수 있지만 TensorFlow 그래프만 갖게 됩니다. 자세한 내용은 `tf.keras.models.load_model` API 문서 및 [Keras 모델 저장 및 로드하기](https://www.tensorflow.org/guide/keras/save_and_serialize#savedmodel_format) 가이드를 참조하세요.

In [ ]:
loaded_model = tf.keras.models.load_model('keras-model')
loaded_model.predict_on_batch(tf.constant([1, 3, 4]))

## GraphDef 및 MetaGraphDef

&lt;a name="graphdef_and_metagraphdef"&gt;

원시 `GraphDef` 또는 `MetaGraphDef`를 TF2에 로드하는 직접적인 방법은 없습니다. 그러나 [`v1.wrap_function`](https://www.tensorflow.org/api_docs/python/tf/compat/v1/wrap_function)을 사용하여 그래프를 TF2 [`concrete_function`](https://tensorflow.org/guide/concrete_function)으로 가져오는 TF1 코드를 변환할 수 있습니다.

먼저 MetaGraphDef를 저장합니다.

In [ ]:
# Save a simple multiplication computation:
with tf.Graph().as_default() as g:
x = tf1.placeholder(tf.float32, shape=[], name='x')
v = tf.Variable(3.0, name='v')
y = tf.multiply(x, v, name='y')
with tf1.Session() as sess:
sess.run(v.initializer)
print(sess.run(y, feed_dict={x: 5}))
s = tf1.train.Saver()
s.export_meta_graph('multiply.pb', as_text=True)
s.save(sess, 'multiply_values.ckpt')

TF1 API를 사용하면 `tf1.train.import_meta_graph`를 사용하여 그래프를 가져오고 값을 복원할 수 있습니다.

In [ ]:
with tf.Graph().as_default() as g:
meta = tf1.train.import_meta_graph('multiply.pb')
x = g.get_tensor_by_name('x:0')
y = g.get_tensor_by_name('y:0')
with tf1.Session() as sess:
meta.restore(sess, 'multiply_values.ckpt')
print(sess.run(y, feed_dict={x: 5}))

그래프를 로드할 때 사용하는 TF2 API는 없지만 즉시 실행 모드에서 실행할 수 있는 구체적인 함수로 그래프를 가져올 수 있습니다.

In [ ]:
def import_multiply():
# Any graph-building code is allowed here.
tf1.train.import_meta_graph('multiply.pb')

# Creates a tf.function with all the imported elements in the function graph.
wrapped_import = tf1.wrap_function(import_multiply, [])
import_graph = wrapped_import.graph
x = import_graph.get_tensor_by_name('x:0')
y = import_graph.get_tensor_by_name('y:0')

# Restore the variable values.
tf1.train.Saver(wrapped_import.variables).restore(
sess=None, save_path='multiply_values.ckpt')

# Create a concrete function by pruning the wrap_function (similar to sess.run).
multiply_fn = wrapped_import.prune(feeds=x, fetches=y)

# Run this function
multiply_fn(tf.constant(5.)) # inputs to concrete functions must be Tensors.

## TensorFlow 1에서 TensorFlow 2로의 변경 사항

&lt;a id="changes_from_tf1_to_tf2"&gt;

이 섹션에서는 TensorFlow 1의 주요 저장 및 로드 용어에 해당하는 TensorFlow 2 용어와 변경 사항을 나열합니다.

### SavedModel

[SavedModel](../../guide/saved_model.ipynb)은 매개변수 및 계산을 사용하여 전체 TensorFlow 프로그램을 저장하는 형식입니다. 여기에는 모델을 실행하기 위해 서비스 플랫폼에서 사용하는 서명이 포함되어 있습니다.

파일 형식 자체는 크게 변경되지 않았으므로 TensorFlow 1 또는 TensorFlow 2 API를 사용하여 SavedModel을 로드하고 제공할 수 있습니다.

**TensorFlow 1과 TensorFlow 2의 차이점**

*서빙* 및 *추론* 사용 사례는 API 변경 사항 외에는 TensorFlow 2에서 업데이트되지 않았습니다. 개선 사항은 SavedModel에서 로드한 *재사용* 및 *구성 모델*에 소개되었습니다.

TensorFlow 2에서 프로그램은 `tf.Variable`이나 `tf.Module` 혹은 고수준 Keras 모델(`tf.keras.Model`) 및 레이어(`tf.keras.layers`)와 같은 객체로 표현됩니다. 세션에 값이 저장된 전역 변수가 더 이상 없으며 이제 그래프가 다른 `tf.function`에 존재합니다. 결과적으로 모델을 내보내는 동안 SavedModel은 각 구성 요소와 함수 그래프를 별도로 저장합니다.

TensorFlow Python API를 사용하여 TensorFlow 프로그램을 작성할 때에는 변수, 함수 및 기타 리소스를 관리하는 객체를 빌드해야 합니다. 일반적으로 이는 Keras API를 사용하여 수행되지만 `tf.Module`을 생성하거나 하위 클래스화하여 객체를 빌드할 수도 있습니다.

Keras 모델(`tf.keras.Model`)과 `tf.Module`는 연결된 변수와 함수를 자동으로 추적합니다. SavedModel은 로드 시 복원할 수 있도록 모듈과 변수와 함수 사이의 이러한 연결을 저장합니다.

### 서명

서명은 SavedModel의 엔드포인트입니다. 서명은 사용자에게 모델을 실행하는 방법과 필요한 입력을 알려줍니다.

TensorFlow 1에서 서명은 입력 및 출력 텐서를 나열함으로써 생성됩니다. TensorFlow 2에서 서명은 *구체적인 함수*를 전달하여 생성됩니다([그래프 및 tf.function 소개](../intro_to_graphs.ipynb) 가이드, 특히 *다형성: 하나의 함수, 많은 그래프* 섹션에서 TensorFlow 함수에 대한 자세한 내용 참조). 간단히 말해서 `tf.function`에서 생성한 구체적인 함수는 다음과 같습니다.

```python
# Option 1: Specify an input signature.
@tf.function(input_signature=[...])
def fn(...):
  ...
  return outputs

tf.saved_model.save(model, path, signatures={
    'name': fn
})
```

```python
# Option 2: Call `get_concrete_function`
@tf.function
def fn(...):
  ...
  return outputs

tf.saved_model.save(model, path, signatures={
    'name': fn.get_concrete_function(...)
})
```

### `Session.run`

TensorFlow 1에서는 텐서 이름을 이미 알고 있는 한 가져온 그래프로 `Session.run`을 호출할 수 있습니다. 이렇게 하면 복원된 변수 값을 검색하거나 서명에서 내보내지 않은 모델의 일부를 실행할 수 있습니다.

TensorFlow 2에서는 가중치 행렬({code 0}kernel{/code 0})과 같은 변수에 직접 액세스할 수 있습니다.

```python
model = tf.Module()
model.dense_layer = tf.keras.layers.Dense(...)
tf.saved_model.save('my_saved_model')
loaded = tf.saved_model.load('my_saved_model')
loaded.dense_layer.kernel
```

또는, 모델 객체에 연결된 `tf.function`을 호출할 수 있습니다(예: `loaded.__call__`).

TF1과 달리 함수의 일부를 추출하고 중간 값에 액세스할 방법이 없습니다. 저장한 객체에 필요한 기능을 *반드시* 모두 내보내야 합니다.


## TensorFlow Serving 마이그레이션 참고사항

SavedModel은 원래 [TensorFlow Serving](https://www.tensorflow.org/tfx/guide/serving)과 함께 작동하도록 제작되었습니다. 이 플랫폼은 분류, 회귀, 예측과 같은 다양한 유형의 예측 요청을 제공합니다.

**TensorFlow 1** API를 사용하면 다음과 같은 유틸로 이러한 유형의 서명을 생성할 수 있습니다.

- `tf.compat.v1.saved_model.classification_signature_def`
- `tf.compat.v1.saved_model.regression_signature_def`
- `tf.compat.v1.saved_model.predict_signature_def`

[분류](https://www.tensorflow.org/tfx/serving/signature_defs#classification_signaturedef)(`classification_signature_def`) 및 [회귀](https://www.tensorflow.org/tfx/serving/signature_defs#regression_signaturedef)(`regression_signature_def`)는 입력과 출력을 제한하므로 입력은 { code4}tf.Example이어야 하고 출력은 `classes`나 `scores`나 `prediction`이어야 합니다. 한편 [예측 서명](https://www.tensorflow.org/tfx/serving/signature_defs#predict_signaturedef)(`predict_signature_def`)에는 제한이 없습니다.

**TensorFlow 2** API로 내보낸 SavedModels은 TensorFlow Serving과 호환되지만 예측 서명만 포함합니다. 분류 및 회귀 서명은 제거되었습니다.

분류 및 회귀 서명을 사용해야 하는 경우 `tf.compat.v1.saved_model.signature_def_utils.MethodNameUpdater`를 사용하여 내보낸 SavedModel을 수정할 수 있습니다.

## 다음 단계

TensorFlow 2의 SavedModels 에 대한 자세한 내용은 다음 가이드를 확인하세요.

- [SavedModel 형식 사용하기](https://www.tensorflow.org/guide/saved_model)
- [Keras 모델 저장 및 로드하기](https://www.tensorflow.org/guide/keras/save_and_serialize)

TensorFlow Hub를 사용하는 경우 다음 가이드가 유용할 수 있습니다.

- [TensorFlow Hub: TensorFlow 1/TensorFlow 2와의 모델 호환성](https://www.tensorflow.org/hub/model_compatibility)
- [TensorFlow Hub를 사용하여 TensorFlow 1에서 TensorFlow 2로 마이그레이션하기](https://www.tensorflow.org/hub/migration_tf2)